In [1]:
import pandas as pd
import os
import glob
from datetime import datetime, timedelta
from google.cloud import bigquery
from google.oauth2 import service_account

Extração e Integração de Dados na Camada Bronze com BigQuery

In [2]:
# Definir o caminho das credenciais
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"C:\Users\Bruno\Downloads\case-saude-6a6b66a8d83a.json"

# Função para extrair dados do CSV
def extrair_dados(arquivo_csv: str) -> pd.DataFrame:
    df = pd.read_csv(arquivo_csv, 
                     encoding='ISO-8859-1', 
                     sep=';',  
                     on_bad_lines='skip', 
                     low_memory=False)
    
    # Tratar o nome da coluna para o formato correto
    df.rename(columns={"#ID_PLANO": "ID_PLANO"}, inplace=True)
    
    return df

# Função para carregar os dados brutos na tabela bronze
def carregar_dados_bronze(df: pd.DataFrame, tabela_bronze: str) -> None:
    client = bigquery.Client()
    
    # Carregar dados no BigQuery
    job = client.load_table_from_dataframe(df, tabela_bronze)
    
    job.result()  # Aguarda a conclusão do carregamento
    print(f"Dados brutos carregados na tabela bronze: {tabela_bronze}")

# Definir o caminho do arquivo CSV
arquivo_csv = r"C:\Users\Bruno\Desktop\projetos_engenharia\Histórico de Planos de Saúde\src\data\HISTORICO_PLANOS.csv"

# Extrair os dados do CSV
df_bronze = extrair_dados(arquivo_csv)

# Carregar os dados para a tabela tb_bronze no dataset bronze
carregar_dados_bronze(df_bronze, 'case-saude.bronze.tb_bronze')


Dados brutos carregados na tabela bronze: case-saude.bronze.tb_bronze


Transformação e Integração de Dados na Camada Ouro com BigQuery

In [78]:
def transformacao_df(df: pd.DataFrame) -> pd.DataFrame:
    # Renomeia as colunas
    df = df.rename(columns={
        'DE_SITUACAO_PRINCIPAL': 'Situacao',
        'ID_PLANO': 'ID_Planos',
        'DT_INICIO_STATUS': 'Data_Inicio'
    })
    
    # Converte a coluna de data para datetime
    df['Data_Inicio'] = pd.to_datetime(df['Data_Inicio'], errors='coerce', dayfirst=True)
    
    # Filtra as colunas desejadas
    df_filtrado = df[['ID_Planos', 'Data_Inicio', 'Situacao']]
    
    # Calcula a data de três meses atrás
    data_tres_meses = datetime.now() - timedelta(days=90)
    
    # Filtra registros dos últimos 3 meses
    df_filtrado = df_filtrado[df_filtrado['Data_Inicio'] >= data_tres_meses]
    
    # Filtra apenas situações "ATIVO" e "CANCELADO"
    df_filtrado = df_filtrado.query("Situacao in ['ATIVO', 'CANCELADO']")
    
    return df_filtrado

# Função para carregar os dados transformados na tabela ouro
def carregar_dados_ouro(df: pd.DataFrame, tabela_ouro: str) -> None:
    client = bigquery.Client()

   # Carregar dados no BigQuery
    job = client.load_table_from_dataframe(df, tabela_ouro)
    
    job.result()  # Aguarda a conclusão do carregamento
    print(f"Dados transformados carregados na tabela ouro: {tabela_ouro}")

# Extrair os dados do CSV
df_bronze = extrair_dados(arquivo_csv)

# Aplicar a transformação
df_ouro = transformacao_df(df_bronze)

# Carregar os dados transformados para a tabela tb_ouro no dataset ouro
carregar_dados_ouro(df_ouro, 'case-saude.ouro.tb_ouro')

Dados transformados carregados na tabela ouro: case-saude.ouro.tb_ouro
